In [4]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
using JuMP, Ipopt
using LinearAlgebra, Plots
import FiniteDiff
import ForwardDiff as FD
using MeshCat
using Test
using Plots

  Activating project at `c:\Users\aphia\Desktop\school\ocrl\QuadraCat`


In [5]:
# include the functions from quadruped.jl
include(joinpath(@__DIR__, "utils", "fmincon.jl"))
include(joinpath(@__DIR__, "utils", "quadruped.jl"))

# this loads in our continuous time dynamics function xdot = dynamics(model, x, u)

initialize_visualizer (generic function with 1 method)

In [7]:
a1 = UnitreeA1() # contains all the model properties for the quadruped

tf = 5
dt = 0.1

t_vec = 0:dt:tf 
N = length(t_vec)
X = [zeros(state_dim(a1)) for i in 1:N]

51-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.

In [8]:
function foot_position(q::AbstractVector, mech::Mechanism, foot="RR")
    state = MechanismState(mech)
    set_configuration!(state, q)

    foot_body = findbody(mech, foot * "_foot")
    world_frame = default_frame(root_body(mech))
    foot_frame = default_frame(foot_body)

    tf = transform(state, world_frame, foot_frame)
    return translation(tf)
end


foot_position (generic function with 2 methods)

In [9]:


function optimize_jump()
    mech = a1.mech
    nq = num_positions(mech)
    nv = num_velocities(mech)
    N = 10   # time steps
    dt = 0.02

    model = Model(Ipopt.Optimizer)
    set_silent(model)

    # Decision variables
    @variable(model, q[1:nq, 1:N])
    @variable(model, v[1:nv, 1:N])
    @variable(model, τ[1:nv, 1:N-1])

    # Initial condition (standing pose)
    q0 = configuration(MechanismState(mech))
    v0 = zeros(nv)
    @constraint(model, q[:, 1] .== q0)
    @constraint(model, v[:, 1] .== v0)

    # Dynamics constraints (Euler integration)
    for k = 1:N-1
        state = MechanismState(mech)
        @NLconstraint(model, [i=1:nq], q[i, k+1] == q[i, k] + dt * v[i, k])
        # acceleration dynamics via implicit FD
        set_configuration!(state, q[:, k])
        set_velocity!(state, v[:, k])
        τk = τ[:, k]
        cache = DynamicsResultCache(mech)
        dynamics!(cache, state, τk)
        a = acceleration(cache)
        @NLconstraint(model, [i=1:nv], v[i, k+1] == v[i, k] + dt * a[i])
    end

    # Foot constraint during STANCE (first 3 timesteps)
    for k = 1:3
        register(model, :foot_pos_z, nq, (x) -> foot_position(x, mech, "RR")[3], autodiff = true)
        @NLconstraint(model, foot_pos_z(q[:, k]) == 0.0)  # on the ground
    end

    # Flight phase (lift-off), foot above ground
    for k = 4:N
        register(model, :foot_pos_z, nq, (x) -> foot_position(x, mech, "RR")[3], autodiff = true)
        @NLconstraint(model, foot_pos_z(q[:, k]) ≥ 0.05)  # minimum 5cm clearance
    end

    # Objective: maximize base height at peak
    register(model, :base_z, nq, x -> x[3], autodiff = true)  # z of floating base
    @objective(model, Max, base_z(q[:, div(N,2)]))

    optimize!(model)

    return value.(q), value.(v), value.(τ)
end


optimize_jump (generic function with 1 method)

In [11]:
q, v, τ = optimize_jump()

MethodError: MethodError: Cannot `convert` an object of type VariableRef to an object of type Float64

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:84
  convert(::Type{T}, !Matched::Gray) where T<:Real
   @ ColorTypes C:\Users\aphia\.julia\packages\ColorTypes\vpFgh\src\conversions.jl:113
  convert(::Type{T}, !Matched::Gray24) where T<:Real
   @ ColorTypes C:\Users\aphia\.julia\packages\ColorTypes\vpFgh\src\conversions.jl:114
  ...


In [10]:
using Plots

function plot_base_z(q_traj)
    z_vals = q_traj[3, :]  # index 3 is base z in floating-base coordinates
    plot(z_vals, xlabel="Time Step", ylabel="Base Height (m)", title="Jump Trajectory")
end


plot_base_z (generic function with 1 method)